In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import clear_session

In [ ]:
# load data
np_data = np.load("data/dataRS.npy")
# ground truth
noise_output = np_data[:,-1].reshape(len(np_data),1)
input = np_data[:,:-1]

# split data
test_split = 0.33
x_train, x_test, y_train, y_test = train_test_split(input, noise_output, test_size=test_split, random_state=42)

### Enter your tag as unique id for saving

In [ ]:
import time
timestr = time.strftime("%d%m%Y")
tag = str("encoderTest"+timestr)

In [ ]:
latent_dim = 3
pca = PCA(n_components=latent_dim,)
dataPCA = pca.fit_transform(x_train)
encoderClassical = np.load("data/ENCdata_autoencoder-18032023.npy")
encoderQuantum1 = np.load("data/ENCdata_quantumAutoencoder2-18032023.npy")
encoderQuantum2 = np.load("data/EENCdata_quantumAutoencoder1-19032023.npy")
print(dataPCA.shape(), encoderClassical.shape(), encoderQuantum1.shape(), encoderQuantum2.shape())

In [ ]:
def plot_metrics(history):
    n = len(history.history.keys())//2
    fig,axs = plt.subplots(1,n, figsize=(18,5))

    for i,[key,val] in enumerate(history.history.items()):
        axs[i%n].plot(history.history[key], lw=4, label=key.replace("_", " "))
    
    for ax in axs:
        ax.set_yscale("log")
        ax.set_xlabel("epoch", fontsize=16)
        ax.legend(fontsize=14)

In [ ]:
#Create a simple ANN
clear_session()
layer_0 = Input(shape=(3,))
layer_1 = Dense(3, activation="relu")
layer_2 = Dense(1, activation="relu")
ann = Sequential([layer_0, layer_1, layer_2])
ann.summary()

In [ ]:
layer_0 = Input(shape=(3,))
layer_1 = Dense(3, activation="relu")
layer_2 = Dense(1, activation="relu")
annPCA = Sequential([layer_0, layer_1, layer_2])
annPCA.compile(optimizer = 'rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])
ann_historyPCA = annPCA.fit(dataPCA, y_train, epochs=250, batch_size=25, validation_split=0.25)
plot_metrics(ann_historyPCA)

In [ ]:
layer_0 = Input(shape=(3,))
layer_1 = Dense(3, activation="relu")
layer_2 = Dense(1, activation="relu")
annAEC = Sequential([layer_0, layer_1, layer_2])
annAEC.compile(optimizer = 'rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])
ann_historyAEC = annAEC.fit(encoderClassical, y_train, epochs=250, batch_size=25, validation_split=0.25)
plot_metrics(ann_historyAEC)

In [ ]:
layer_0 = Input(shape=(3,))
layer_1 = Dense(3, activation="relu")
layer_2 = Dense(1, activation="relu")
annAEQ1 = Sequential([layer_0, layer_1, layer_2])
annAEQ1.compile(optimizer = 'rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])
ann_historyAEQ1 = annAEQ1.fit(encoderQuantum1, y_train, epochs=250, batch_size=25, validation_split=0.25)
plot_metrics(ann_historyAEQ1)

In [ ]:
layer_0 = Input(shape=(3,))
layer_1 = Dense(3, activation="relu")
layer_2 = Dense(1, activation="relu")
annAEQ2 = Sequential([layer_0, layer_1, layer_2])
annAEQ2.compile(optimizer = 'rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])
ann_historyAEQ2 = annAEQ2.fit(encoderQuantum2, y_train, epochs=250, batch_size=25, validation_split=0.25)
plot_metrics(ann_historyAEQ2)

In [ ]:
fig,axs = plt.subfigure(2,2)
axs[0,0].plot(ann_historyPCA.history['loss'], lw=4, label='loss')
axs[0,0].plot(ann_historyPCA.history['val_loss'], lw=4, label='val_loss')
axs[0,0].set_xlabel("epoch", fontsize=16)
axs[0,0].set_title("Neuronal Network training on PCA data")
axs[0,0].set_legend(fontsize=14)
#-------------------------------
axs[0,1].plot(ann_historyAEC.history['loss'], lw=4, label='loss')
axs[0,1].plot(ann_historyAEC.history['val_loss'], lw=4, label='val_loss')
axs[0,1].set_xlabel("epoch", fontsize=16)
axs[0,1].set_title("Neuronal Network training on classical compressed data")
axs[0,1].set_legend(fontsize=14)
#-------------------------------
axs[1,0].plot(ann_historyAEQ1.history['loss'], lw=4, label='loss')
axs[1,0].plot(ann_historyAEQ1.history['val_loss'], lw=4, label='val_loss')
axs[1,0].set_xlabel("epoch", fontsize=16)
axs[1,0].set_title("Neuronal Network training on quantum compressed data, version 1")
axs[1,0].set_legend(fontsize=14)
#-------------------------------
axs[1,1].plot(ann_historyAEQ2.history['loss'], lw=4, label='loss')
axs[1,1].plot(ann_historyAEQ2.history['val_loss'], lw=4, label='val_loss')
axs[1,1].set_xlabel("epoch", fontsize=16)
axs[1,1].set_title("Neuronal Network training on quantum compressed data, version 2")
axs[1,1].set_legend(fontsize=14)

plt.show()
fig.savefig("archive/history-"+tag+".png")

In [ ]:
annPCA.save("archive/model-"+tag+"PCA.h5")
annPCA.save_weights("archive/weights-"+tag+"PCA.h5")
config = annPCA.get_config()
np.save("archive/config-"+tag+"PCA.npy", config)
#------------------
annAEC.save("archive/model-"+tag+"AEC.h5")
annAEC.save_weights("archive/weights-"+tag+"AEC.h5")
config = annAEC.get_config()
np.save("archive/config-"+tag+"AEC.npy", config)
#------------------
annAEQ1.save("archive/model-"+tag+"AEQ1.h5")
annAEQ1.save_weights("archive/weights-"+tag+"AEQ1.h5")
config = annAEQ1.get_config()
np.save("archive/config-"+tag+"AEQ1.npy", config)
#------------------
annAEQ2.save("archive/model-"+tag+"AEQ2.h5")
annAEQ2.save_weights("archive/weights-"+tag+"AEQ2.h5")
config = annAEQ2.get_config()
np.save("archive/config-"+tag+"AEQ2.npy", config)